In [1]:
import os 

import torch
import torch.nn as nn
import torchaudio

import pandas as pd
import torchmetrics as torchmetrics

str(torchaudio.get_audio_backend())

'sox_io'

## Download the dataset 
As part of HW2, you will work with the audio dataset. Download the dataset by the following [link](https://urbansounddataset.weebly.com/download-urbansound8k.html) and remember a path to it. You will have to train a sound classification model, explore different audio transformation methods and try different model architectures.

In [2]:
class UrbanSoundDataset(torch.utils.data.Dataset):
    def __init__(self,
                 annotations_file,
                 audio_dir,
                 transformation,
                 target_sample_rate,
                 num_samples,
                 device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

## Defining hyperparameters

In [3]:
# Path to the dataset 
ANNOTATIONS_FILE = "datasets/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = "datasets/UrbanSound8K/audio"

# audio signal sample rate 
SAMPLE_RATE = 22050
# max number of samples in audio
NUM_SAMPLES = 22050

# optimizer learning rate
LEARNING_RATE = 1e-5
# number of train epochs
EPOCHS = 10
# number of samples in each batch size
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# defining audio preprocessing function. Refer to torchaudio.transforms to 
# check Spectrogram(), MFCC() and LFCC() transfrormations. Retrain the model
# with each transformation and write your metrics resulst and conclusaions.
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
)

## Define the dataset

In [5]:
# defining dataset class with 
dataset = UrbanSoundDataset(
    ANNOTATIONS_FILE,
    AUDIO_DIR,
    mel_spectrogram,
    SAMPLE_RATE,
    NUM_SAMPLES,
    device
)

# calculating validation split sizes
train_size = int(len(dataset) * 0.8)
val_test_size = len(dataset) - train_size

val_size = val_test_size // 2
test_size = val_test_size - val_size

print('Dataset len: ', len(dataset))
print(f'Train split: {train_size} | Validation split: {val_size} | Test split: {test_size}')

# splitting original dataset into train and val_test sets
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size + test_size]
)
# splitting val_test set into val dataset and test dataset
val_dataset, test_dataset = torch.utils.data.random_split(
    val_dataset, [val_size, test_size]
)

Dataset len:  8732
Train split: 6985 | Validation split: 873 | Test split: 874


In [6]:
signal, label = dataset[0]

## Model

Define your model architecture here. 

1) Build a baseline model that consists of consecutive convolution blocks. Flatten the convolution output and pass through the linear layer with softmax activation function, to obtain class distributions

2) Add the BatchNorm layer after each convolution block and compare the results with the baseline model

3) Add a Dropout layer after each BatchNorm block and compare the results

4) Try different parameters for the blocks:
 - Conv layer: out_channels, kernel_size, stride
 - Dropout: p
 
 and compare the results

In [7]:
# Class CLModel to define your model architecture and computational graph
class CLModel(nn.Module):
    def __init__(self):
        super().__init__()
        ################

        # YOUR CODE HERE
        # define you convolution based classification model
        # 4 conv blocks / flatten / linear / softmax

        ################

        self.conv1 = nn.Sequential(
                nn.Conv2d(
                    in_channels=1,
                    out_channels=16,
                    kernel_size=(3,3),
                    padding=2
                ),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=(3,3),
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=(3,3),
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=(3,3),
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128 * 5 * 4, 10)
        self.softmax = nn.Softmax(dim=1)

        # #First Conv Block
        # self.conv1 = nn.Conv2d(1, 16, kernel_size=(3,3), padding=2)
        # self.relu1 = nn.ReLU()
        # self.pool1 = nn.MaxPool2d(2,2)
        # #self.bn1 = nn.BatchNorm2d(8)
        # #self.dropout1 = nn.Dropout(0.2)
        #
        # #Second Conv Block
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=(3,3), padding=2)
        # self.relu2 = nn.ReLU()
        # self.pool2 = nn.MaxPool2d(2,2)
        # #self.bn2 = nn.BatchNorm2d(8)
        # #self.dropout2 = nn.Dropout(0.2)
        #
        # #Third Conv Block
        # self.conv3 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=2)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(2,2)
        # #self.bn3 = nn.BatchNorm2d(8)
        # #self.dropout3 = nn.Dropout(0.2)
        #
        # #Fourth Conv Block
        # self.conv4 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=2)
        # self.relu4 = nn.ReLU()
        # self.pool4 = nn.MaxPool2d(2,2)
        # #self.bn4 = nn.BatchNorm2d(8)
        # #self.dropout4 = nn.Dropout(0.2)
        #
        # self.flatten = nn.Flatten()
        # self.lin = nn.Linear(in_features=128 * 5 * 4, out_features=10)
        # self.softmax = nn.Softmax(dim=1)
        #
        #
        # self.conv = nn.Sequential(
        #     self.conv1,
        #     self.relu1,
        #     self.pool1,
        #
        #     #self.bn1,
        #     #self.dropout1,
        #
        #     self.conv2,
        #     self.relu2,
        #     self.pool2,
        #     #self.bn2,
        #     #self.dropout2,
        #
        #     self.conv3,
        #     self.relu3,
        #     self.pool3,
        #     #self.bn3,
        #     #self.dropout3,
        #
        #     self.conv4,
        #     self.relu4,
        #     self.pool4,
        #     #self.bn4,
        #     #self.dropout4,
        #
        # )


    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        ################

        # YOUR CODE HERE
        # define the logic of your computational graph

        ################
        # Run the convolutional blocks
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return predictions

        # data = data.view(data.shape[0], -1)
        # data = self.lin(data)
        # data = self.softmax(data)

        #return data


In [8]:
cnn = CLModel()
from torchsummary import summary
summary(cnn, (1, 64, 44))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 33, 23]          --
|    └─Conv2d: 2-1                       [-1, 16, 66, 46]          160
|    └─ReLU: 2-2                         [-1, 16, 66, 46]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 33, 23]          --
├─Sequential: 1-2                        [-1, 32, 17, 12]          --
|    └─Conv2d: 2-4                       [-1, 32, 35, 25]          4,640
|    └─ReLU: 2-5                         [-1, 32, 35, 25]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 17, 12]          --
├─Sequential: 1-3                        [-1, 64, 9, 7]            --
|    └─Conv2d: 2-7                       [-1, 64, 19, 14]          18,496
|    └─ReLU: 2-8                         [-1, 64, 19, 14]          --
|    └─MaxPool2d: 2-9                    [-1, 64, 9, 7]            --
├─Sequential: 1-4                        [-1, 128, 5, 4]           --
|    └─

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 33, 23]          --
|    └─Conv2d: 2-1                       [-1, 16, 66, 46]          160
|    └─ReLU: 2-2                         [-1, 16, 66, 46]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 33, 23]          --
├─Sequential: 1-2                        [-1, 32, 17, 12]          --
|    └─Conv2d: 2-4                       [-1, 32, 35, 25]          4,640
|    └─ReLU: 2-5                         [-1, 32, 35, 25]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 17, 12]          --
├─Sequential: 1-3                        [-1, 64, 9, 7]            --
|    └─Conv2d: 2-7                       [-1, 64, 19, 14]          18,496
|    └─ReLU: 2-8                         [-1, 64, 19, 14]          --
|    └─MaxPool2d: 2-9                    [-1, 64, 9, 7]            --
├─Sequential: 1-4                        [-1, 128, 5, 4]           --
|    └─

## Train and evaluation functions

In [11]:
def create_data_loader(data, batch_size:int, shuffle:bool=False) -> torch.utils.data.DataLoader:
    ''' creating dataloader  
            Arguments:
                batch_size: int
                    number of samples to process for the model
                shuffle: bool
                    whether to shuffle dataset. 
    '''
    dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    return dataloader

def train_single_epoch(
    model:nn.Module, 
    data_loader:torch.utils.data.DataLoader, 
    loss_fn:torch.nn.modules.loss, 
    optimiser:torch.optim,
    metrics,
    device:torch.device
):
    ''' method to perform single epoch of training '''

    rolling_loss = 0.
    #rolling_metric = 0.
    rolling_metric = metrics()
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)
        ################

        # YOUR CODE HERE 
        optimiser.zero_grad()
        # step 1: pass input tensor through the model
        output = cnn(input)

        # step 2: compute loss
        loss = loss_fn(output, target)

        # step 3: update optimizer and do a backward for the loss function
        loss.backward()
        optimiser.step()

        #print(f"loss: {loss.item()}")

        _, prediction = torch.max(output,1)
        # step 5 (final): compute train epoch loss and metrics
        rolling_loss += loss.item()
        rolling_metric(prediction, target)
        #rolling_metric += (prediction == target ).float().sum()
        #metric = metrics(prediction, target)
        #print(f"metric: {metric}")
        #rolling_metric += metric

    ################
    total_rolling_metric = rolling_metric.compute()
    print(f'Train loss: {rolling_loss} | Train metric: {total_rolling_metric}')


def evaluate(
    model:nn.Module,
    val_dataloader:torch.utils.data.DataLoader, 
    loss_fn:torch.nn.modules.loss, 
    metrics, # define you type here  
    device:torch.device):
    ''' method to perform evaluation step '''

    rolling_loss = 0.
    rolling_metric = metrics()
    for input, target in val_dataloader: 
        input, target = input.to(device), target.to(device)
        ################

        # YOUR CODE HERE 

        # step 1: pass input tensor through the model
        output = cnn(input)
        # step 2: compute loss and metrics
        loss = loss_fn(output, target)
        _, prediction = torch.max(output,1)

        rolling_loss += loss.item()
        #rolling_metric += metrics(prediction, target)
        rolling_metric(prediction, target)
    ################
    total_rolling_metric = rolling_metric.compute()

    print(f'Validation loss: {rolling_loss} | Validation metric: {total_rolling_metric}')
    

def train(model, train_dataloader, val_dataloader, loss_fn, metrics, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, train_dataloader, loss_fn, optimiser, metrics, device)
        evaluate(model, val_dataloader, loss_fn, metrics, device)
    print("Finished training")


cnn = CLModel()
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(cnn.parameters(),lr=LEARNING_RATE)
metrics = torchmetrics.Accuracy

train_dataloader = create_data_loader(train_dataset, BATCH_SIZE)
val_dataloader = create_data_loader(val_dataset, BATCH_SIZE)
test_dataloader = create_data_loader(test_dataset, BATCH_SIZE)


# train model
train(cnn, train_dataloader, val_dataloader, loss_fn, metrics, optimiser, device, EPOCHS)

# save model
torch.save(cnn.state_dict(), "feedforwardnet.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Epoch 1
Train loss: 987.5744919776917 | Train metric: 0.19026485085487366
Validation loss: 120.64897656440735 | Validation metric: 0.2955326437950134
Epoch 2
Train loss: 938.9268516302109 | Train metric: 0.3408733010292053
Validation loss: 117.39090812206268 | Validation metric: 0.36311569809913635
Epoch 3
Train loss: 921.0997983217239 | Train metric: 0.3937007784843445
Validation loss: 116.56013870239258 | Validation metric: 0.3791523575782776
Epoch 4
Train loss: 912.2578115463257 | Train metric: 0.41760915517807007
Validation loss: 116.15880858898163 | Validation metric: 0.40091636776924133
Epoch 5
Train loss: 904.7090142965317 | Train metric: 0.4357909858226776
Validation loss: 115.44562494754791 | Validation metric: 0.41237112879753113
Epoch 6
Train loss: 901.6253417730331 | Train metric: 0.4406585395336151
Validation loss: 114.60337555408478 | Validation metric: 0.42611682415008545
Epoch 7
Train loss: 895.0773813724518 | Train metric: 0.45397278666496277
Validation loss: 114.42016

In [24]:
# YOUR CODE HERE
# Define your loss function, optimization algorithm, and a metric function(s)
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(cnn.parameters(),lr=0.001)
metrics = torchmetrics.functional.accuracy

train_dataloader = create_data_loader(train_dataset, BATCH_SIZE)
val_dataloader = create_data_loader(val_dataset, BATCH_SIZE)
test_dataloader = create_data_loader(test_dataset, BATCH_SIZE)


# train model
train(cnn, train_dataloader, val_dataloader, loss_fn, metrics, optimiser, device, EPOCHS)

# save model
torch.save(cnn.state_dict(), "feedforwardnet.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Epoch 1


C:\Users\Olexandr\AppData\Local\Temp/ipykernel_3424/4097236304.py:87: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  data = self.softmax(data)


RuntimeError: The size of tensor a (10) must match the size of tensor b (16) at non-singleton dimension 1

In [ ]:
################

# YOUR CODE HERE
# evaluate your model on test split

################

## Results
1) Write a litter summary of the audio transformation methods (1-2 sentences), and explore them on your own

2) Create a table with the model performance comparison. The table should include short model summary (ex. 4xConvBlocks - Flatten - Linear - Softmax), comments about hyperparameters (ex. ConvBlocks: [64, 64, 64, 64] - Dropout: 0.2, etc.) and metrics values for validation and test datasets.

3) Train baseline model with 4x ConvBlocks - Flatten - Linear - Softmax

4) Train baseline model with BatchNorm

5) Train baseline model with BatchNorm and Dropout

6) Train 5 model models with different hyperparameters

7) Add all models to a table 

8) Write an explanation to the table and indicate the best model parameters and architecture